<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

In [1]:
import numpy as np
import pydicom
import matplotlib.pyplot as plt
import math
import pandas as pd
from skimage.feature import greycomatrix
from skimage.feature import greycoprops
from skimage.measure import shannon_entropy
import pandas as pd
import os
from skimage.feature import greycomatrix as gray
from skimage.feature import greycoprops as props
import cv2 as cv
import pandas as pd
import numpy as np
import tensorflow as tf
import cv2 as cv
import matplotlib.pyplot as plt
from sklearn import linear_model
from sklearn.preprocessing import PolynomialFeatures
import pydicom
from sklearn.metrics import roc_auc_score
from copy import deepcopy
from sklearn.utils import shuffle
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn import tree
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense
import seaborn as sns
import os
os.environ['KMP_DUPLICATE_LIB_OK']='True'
import tifffile as tiff

In [2]:
#For this function, pass the original image without normalization
def calc_entropy(img):
    image_size = img.size
    values_count = []
    for i in range(np.min(img), np.max(img) + 1):
        occurrences = np.count_nonzero(img == i)
        values_count.append(occurrences)
    image_size -= values_count[0]
    values_count = [i/image_size for i in values_count]
    s = 0
    for i in range(1, len(values_count)):
        if values_count[i] == 0:
            pass
        else:
            s = -1*values_count[i]*math.log2(values_count[i]) + s
    return s

In [3]:
def graycoprops(glcm):
    levels = glcm.shape[0]
    I, J = np.ogrid[0:levels, 0:levels]
    
    contrast_weights = (I - J) ** 2
    
    dissimilarity_weights = np.abs(I - J)
    
    homogeneity_weights = 1. / (1. + (I - J) ** 2)
    
    diff_i = I - np.apply_over_axes(np.sum, (I*glcm), axes = (0,1))[0, 0]
    diff_j = J - np.apply_over_axes(np.sum, (J*glcm), axes = (0,1))[0, 0]
    std_i = np.sqrt(np.apply_over_axes(np.sum, (glcm * (diff_i) ** 2),
                                           axes=(0, 1))[0, 0])
    std_j = np.sqrt(np.apply_over_axes(np.sum, (glcm * (diff_j) ** 2),
                                           axes=(0, 1))[0, 0])
    cov = np.apply_over_axes(np.sum, (glcm * (diff_i * diff_j)),
                                 axes=(0, 1))[0, 0]
    
    correlation = cov / (std_i * std_j)
    
    #Angular Second Moment
    asm = np.apply_over_axes(np.sum, (glcm ** 2), axes = (0, 1))[0, 0]
    
    #Energy
    energy = np.sqrt(asm)
    
    #Contrast
    contrast = np.apply_over_axes(np.sum, (glcm*contrast_weights), 
                                  axes = (0,1))[0][0]
    
    #Dissimilarity
    dissimilarity = np.apply_over_axes(np.sum, (glcm*dissimilarity_weights), 
                                  axes = (0,1))[0][0]
    
    #Homogeneity
    homogeneity = np.apply_over_axes(np.sum, (glcm*homogeneity_weights), 
                                  axes = (0,1))[0][0]
    
    return asm, energy, contrast, dissimilarity, homogeneity, correlation

In [4]:
#Creation of GLCM matrix for excluding the background

#Pass image and distance to be calculated
def graycomatrix_thresh(img, distance):
    max_val = np.amax(img)
    min_val = np.amin(img)
    img = img + min_val*-1
    rows = img.shape[0]
    cols = img.shape[1]
    max_val = np.amax(img)
    min_val = np.amin(img)
    size_glcm = 0
    if min_val == 0:
        size_glcm = max_val + 1
    elif min_val < 0:
        size_glcm = max_val + 1 + min_val*-1
    else:
        size_glcm = max_val
    glcm = np.zeros((size_glcm, size_glcm))
    f = img.flatten()
    l = len(f)
    c = -1
    while(c < l-1):
        c += 1
        if f[c] == min_val:
            pass
        else:
            if c%cols >= cols-distance:
                pass
            else:
                glcm[f[c]][f[c + distance]]+= 1
    glcm = glcm/(np.apply_over_axes(np.sum, glcm, axes = (0, 1)))
    return glcm

In [5]:
def calculate_textures(pp_img, distance, patient_no, status, count):
    glcm = graycomatrix_thresh(pp_img, distance)
    cols = ['Patient', 'Slice no', 'Distance', 'ASM', 'Energy', 'Contrast', 'Dissimilarity', 'Homogeneity', 'Correlation', 'Entropy', 'Status']
    texture_values = list(graycoprops(glcm))
    texture_values.insert(0, distance)
    texture_values.insert(0, count)
    texture_values.insert(0, patient_no)
    texture_values.append(calc_entropy(pp_img))
    texture_values.append(status)
    mydict = {k:v for (k, v) in zip(cols, texture_values)}
    return mydict

In [6]:
def preprocessing(path):
    DICOM = pydicom.dcmread(path)
    img = DICOM.pixel_array
    size = img.shape[0]
    
    offset=1024
    gray1=img.astype(np.int16)-offset
    gray2=np.zeros((size,size),dtype=np.int16)

    for i in range(size):
        for j in range(size):
            if gray1[i][j]>=-200 and gray1[i][j]<=200:
                gray2[i][j]=255
            else:
                pass

    gray_ex=np.zeros([size,size],dtype=np.uint16)
    for i in range(size):
        for j in range(size):
            if gray2[i][j]==255:
                gray_ex[i][j]=img[i][j]
            else:
                pass    

    gray_ex= cv.medianBlur(gray_ex,5)
    
    return gray_ex

In [7]:
def dataset_tool(basepath):
    subdirectory2 = []
    for i in os.listdir(basepath):
        if os.path.isdir(os.path.join(basepath, i)):
            subdirectory1 = os.path.join(basepath, i)
            for j in (os.listdir(subdirectory1)):
                if os.path.isdir(os.path.join(subdirectory1, j)):
                    subdirectory2.append(os.path.join(subdirectory1, j))
    directories = sorted(subdirectory2)
    counter = 0
    d = {}
    for i, j in zip(range(0, len(directories) - 1, 2), range(1, len(directories), 2)):
        false = directories[i]
        true = directories[j]
        true_directories = []
        false_directories = []
        for entry in os.listdir(true):
            if os.path.isfile(os.path.join(true, entry)):
                if entry != '.DS_Store':
                    true_directories.append(os.path.join(true, entry))

        for entry in os.listdir(false):
            if os.path.isfile(os.path.join(false, entry)):
                if entry != '.DS_Store':
                    false_directories.append(os.path.join(false, entry))

        counter += 1
        d[counter] = {}
        d[counter]['true'] = true_directories
        d[counter]['false'] = false_directories
    no_patients = d.keys()
    cols = ['Patient', 'Slice no', 'Distance', 'ASM', 'Energy', 'Contrast', 'Dissimilarity', 'Homogeneity', 'Correlation', 'Entropy', 'Status']
    count = 0
    df = pd.DataFrame(columns = cols)
    df_true = pd.DataFrame(columns = cols)
    df_false = pd.DataFrame(columns = cols)
    
    for i in no_patients:
        temp1, temp2 = d[i].keys()
        for j, k in zip(d[i][temp1], d[i][temp2]):
            count += 1
            pp_true = preprocessing(j) 
            pp_false = preprocessing(k)
            
            df = df.append(calculate_textures(pp_true, 1, i, 1, count), ignore_index = True)
            df = df.append(calculate_textures(pp_false, 1, i, 0, count), ignore_index = True)
            
            df = df.append(calculate_textures(pp_true, 2, i, 1, count), ignore_index = True)
            df = df.append(calculate_textures(pp_false, 2, i, 0, count), ignore_index = True)
            
            df = df.append(calculate_textures(pp_true, 3, i, 1, count), ignore_index = True)
            df = df.append(calculate_textures(pp_false, 3, i, 0, count), ignore_index = True)
        display(df)
    return df

In [8]:
training = '/Users/rt/Desktop/sharmaji'
df = dataset_tool(training)
df.to_csv('/Users/rt/Desktop/sharmaji.csv')

,Patient,Slice no,Distance,ASM,Energy,Contrast,Dissimilarity,Homogeneity,Correlation,Entropy,Status
0,1.0,1.0,1.0,0.000426,0.020648,28424.728525,37.977875,0.315105,0.504229,7.909620,1.0
1,1.0,1.0,1.0,0.000307,0.017527,22809.630525,37.422471,0.225428,0.539505,7.779515,0.0
2,1.0,1.0,2.0,0.000225,0.015013,58036.432055,73.172447,0.177097,0.423882,7.909620,1.0
3,1.0,1.0,2.0,0.000172,0.013096,46250.780856,66.756498,0.135756,0.455201,7.779515,0.0
4,1.0,1.0,3.0,0.000184,0.013566,86443.744296,105.789695,0.117290,0.383711,7.909620,1.0
5,1.0,1.0,3.0,0.000140,0.011838,69313.545486,93.438911,0.095103,0.397527,7.779515,0.0
6,1.0,2.0,1.0,0.000308,0.017564,22275.596267,36.237089,0.250431,0.539872,7.855264,1.0
7,1.0,2.0,1.0,0.000360,0.018978,18494.257959,29.369515,0.274155,0.525848,7.787111,0.0
8,1.0,2.0,2.0,0.000152,0.012341,44771.749237,65.080319,0.146005,0.445162,7.855264,1.0
9,1.0,2.0,2.0,0.000173,0.013163,37750.420660,54.628549,0.157209,0.405808,7.787111,0.0
